# Component Parts

> System for handling component parts (e.g., card-body, modal-box)

In [ ]:
#| default_exp core.parts

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union
from dataclasses import dataclass, field
from fasthtml.common import *
from cjm_tailwind_utils.all import TailwindBuilder
from cjm_fasthtml_daisyui.core.elements import create_element
from cjm_fasthtml_daisyui.core.types import (
    DaisyComponentType,
    ComponentProtocol, CSSContributor
)

## Component Part Definition

A component part represents a child element of a daisyUI component.

In [ ]:
#| export
@dataclass
class ComponentPart:
    """Represents a part of a component (e.g., card-body, modal-box).
    
    Component parts are child elements that have specific styling
    within their parent component context.
    """
    name: str  # Part name (e.g., 'body', 'title', 'actions')
    parent_component: Union[str, DaisyComponentType]  # Parent component name or enum
    required: bool = False  # Whether this part is required
    tag: str = "div"  # Default HTML tag for this part
    
    def __post_init__(self):
        """Validate parent_component if it's a string."""
        if isinstance(self.parent_component, str):
            # Check if the string matches any DaisyComponentType value
            valid_values = {comp.value for comp in DaisyComponentType}
            if self.parent_component not in valid_values:
                # It might be a valid component name that doesn't match the enum value
                # For now, we'll allow it but could add stricter validation later
                pass
    
    def class_name(
        self
    ) -> str:  # The complete class name for this part (e.g., 'card-body')
        """Return the full class name for this part."""
        # Handle both string and enum types
        parent_name = (
            self.parent_component.value 
            if isinstance(self.parent_component, DaisyComponentType) 
            else self.parent_component
        )
        return f"{parent_name}-{self.name}"

## HasParts Mixin

Mixin class for components that have child parts.

In [ ]:
#| export
class HasParts:
    """Mixin for components that have child parts.
    
    This mixin provides functionality for components like cards, modals,
    and other complex components that have defined child elements.
    
    Note: Classes using this mixin should typically also implement the
    ComponentProtocol to ensure they provide the full component interface.
    Components that contribute CSS classes should also implement CSSContributor.
    """
    
    @classmethod
    def parts(
        cls  # The class implementing HasParts
    ) -> Dict[str, ComponentPart]:  # Dictionary mapping part names to ComponentPart objects
        """Return all available parts for this component.
        
        Subclasses should override this to define their parts.
        """
        return {}
    
    def part(
        self,
        name: str,  # The part name (must be defined in parts())
        *children: FT,  # Child elements for this part
        **attrs: Any  # HTML attributes for the part
    ) -> FT:  # FastHTML element configured as the specified part
        """Create a component part element."""
        parts = self.parts()
        if name not in parts:
            raise ValueError(f"Unknown part '{name}' for {self.__class__.__name__}")
            
        part = parts[name]
        
        # Extract and merge classes
        cls: str = attrs.pop("cls", "")
        if cls:
            # Use TailwindBuilder to merge classes
            tb = TailwindBuilder()
            tb.merge(part.class_name(), cls)
            attrs["class"] = tb.build()
        else:
            attrs["class"] = part.class_name()
        
        # Get the tag to use
        tag: str = attrs.pop("tag", part.tag)
        
        # Use the shared create_element utility
        return create_element(tag, *children, **attrs)

## Usage Examples

Examples showing how to use ComponentPart with proper type safety:

In [ ]:
# Example: Creating component parts with DaisyComponentType enum
card_body = ComponentPart(
    name="body",
    parent_component=DaisyComponentType.CARD,
    required=False,
    tag="div"
)

# The class name is generated correctly from the enum
print(f"Card body class: {card_body.class_name()}")

# Example: Creating parts with string (for backwards compatibility)
modal_box = ComponentPart(
    name="box",
    parent_component="modal",  # Still works with strings
    required=True,
    tag="div"
)

print(f"Modal box class: {modal_box.class_name()}")

Card body class: card-body
Modal box class: modal-box


In [ ]:
# Example: Using HasParts mixin with proper typing
class ExampleCard(HasParts):
    """Example card component demonstrating HasParts usage."""
    
    @classmethod
    def parts(cls) -> Dict[str, ComponentPart]:
        """Define the parts for a card component."""
        return {
            "body": ComponentPart(
                name="body",
                parent_component=DaisyComponentType.CARD,
                required=False,
                tag="div"
            ),
            "title": ComponentPart(
                name="title",
                parent_component=DaisyComponentType.CARD,
                required=False,
                tag="h2"
            ),
            "actions": ComponentPart(
                name="actions",
                parent_component=DaisyComponentType.CARD,
                required=False,
                tag="div"
            )
        }
    
    def render(self) -> FT:
        """Render the card with typed parts."""
        return Div(
            self.part("title", "Card Title"),
            self.part("body", "Card content goes here"),
            self.part("actions", 
                Button("Save", cls="btn-primary"),
                Button("Cancel", cls="btn-ghost")
            ),
            cls="card"
        )

# Create and demonstrate the card
card = ExampleCard()
# This would render the card structure with proper part classes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()